# Proyek Analisis Data: Air Quality Dataset
- **Nama:** muhammad fikri
- **Email:** diqie.muharrom@gmail.com
- **ID Dicoding:** 4519796

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1: Bagaimana tren rata-rata harian PM2.5 dari waktu ke waktu?
- Pertanyaan 2: Pada jam berapa PM2.5 cenderung paling tinggi?

## Import Semua Packages/Library yang Digunakan

In [ ]:
from pathlib import Path
import io
import re
import zipfile
from http.cookiejar import CookieJar
from urllib.parse import urlencode
from urllib.request import HTTPCookieProcessor, Request, build_opener

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

## Data Wrangling

### Gathering Data

Tahap ini berfokus pada pengumpulan **Air Quality Dataset**. Notebook akan:
- Mengutamakan pembacaan file lokal di folder `data/`.
- Jika belum tersedia, notebook akan mengunduh dataset dari tautan Google Drive yang disediakan dan mengekstrak file CSV (jika berupa ZIP).
- Data akan dipersiapkan agar analisis tren berbasis waktu lebih mudah.


In [ ]:
DATASET_FILE_ID = "1RhU3gJlkteaAQfyn9XOVAz7a5o1-etgr"
DATASET_VIEW_URL = f"https://drive.google.com/file/d/{DATASET_FILE_ID}/view"

data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

def _download_google_drive_file(file_id: str) -> bytes:
    jar = CookieJar()
    opener = build_opener(HTTPCookieProcessor(jar))
    base_url = "https://drive.google.com/uc"
    first_url = f"{base_url}?{urlencode({'export': 'download', 'id': file_id})}"
    req = Request(first_url, headers={"User-Agent": "Mozilla/5.0"})
    with opener.open(req) as resp:
        content_type = resp.headers.get("Content-Type", "")
        body = resp.read()
    if "text/html" not in content_type.lower():
        return body
    text = body.decode("utf-8", errors="ignore")
    match = re.search(r"confirm=([0-9A-Za-z_]+)", text)
    if match is None:
        raise RuntimeError("Gagal mengunduh dataset dari Google Drive. Pastikan tautan dapat diakses publik.")
    confirm_token = match.group(1)
    confirm_url = f"{base_url}?{urlencode({'export': 'download', 'confirm': confirm_token, 'id': file_id})}"
    req2 = Request(confirm_url, headers={"User-Agent": "Mozilla/5.0"})
    with opener.open(req2) as resp2:
        return resp2.read()

def _extract_zip_bytes(zip_bytes: bytes, out_dir: Path) -> None:
    with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zf:
        for member in zf.namelist():
            if member.endswith("/"):
                continue
            if not member.lower().endswith(".csv"):
                continue
            target_name = Path(member).name
            out_path = out_dir / target_name
            with zf.open(member) as src, open(out_path, "wb") as dst:
                dst.write(src.read())

def _looks_like_air_quality_csv(p: Path) -> bool:
    try:
        sample = pd.read_csv(p, nrows=50)
    except Exception:
        return False

    cols = {str(c).strip().lower() for c in sample.columns}
    has_time = any(("date" in c) or ("time" in c) or (c in {"datetime", "timestamp", "utc"}) for c in cols) or {"year", "month", "day", "hour"}.issubset(cols)
    has_pollutant = any(("pm2.5" in c) or ("pm25" in c) or ("pm2_5" in c) or ("pm10" in c) or (c in {"so2", "no2", "o3", "co"}) for c in cols)
    return has_time and has_pollutant

def _find_air_quality_csv(search_dir: Path) -> Path | None:
    for p in sorted(search_dir.glob("*.csv")):
        if _looks_like_air_quality_csv(p):
            return p
    return None

air_csv = _find_air_quality_csv(data_dir)
legacy_dir = Path("submission/data")
if air_csv is None and legacy_dir.exists():
    air_csv = _find_air_quality_csv(legacy_dir)

if air_csv is None:
    raw_bytes = _download_google_drive_file(DATASET_FILE_ID)
    if raw_bytes[:4] == b"PK\x03\x04":
        _extract_zip_bytes(raw_bytes, data_dir)
        air_csv = _find_air_quality_csv(data_dir)
    else:
        out_path = data_dir / "air_quality_raw.csv"
        out_path.write_bytes(raw_bytes)
        air_csv = out_path

if air_csv is None:
    raise FileNotFoundError("Air Quality dataset tidak ditemukan setelah proses download/ekstraksi.")

air_df = pd.read_csv(air_csv)
print("Sumber dataset:", DATASET_VIEW_URL)
air_df.head()

**Insight:**
- Data berhasil dimuat dan siap dianalisis berbasis waktu.
- Tahap berikutnya memastikan kualitas data dan menyiapkan kolom waktu untuk agregasi.

### Assessing Data

Tahap assessing dilakukan untuk mengecek kualitas data sebelum dianalisis lebih lanjut, meliputi:
- Struktur data (tipe kolom dan jumlah baris) menggunakan `info()`.
- Missing value dan duplikasi untuk memastikan data bersih.
- Ringkasan statistik awal menggunakan `describe()` untuk memahami rentang nilai dan anomali.


In [ ]:
print("Info dataset:")
print(air_df.info())

print("\nMissing values (top 20 kolom):")
print(air_df.isna().sum().sort_values(ascending=False).head(20))

print("\nDuplikasi baris:", air_df.duplicated().sum())

air_df.describe(include="all")

**Insight:**
- Kolom waktu menjadi kunci untuk analisis tren polusi.
- Missing value pada kolom polutan umum terjadi dan perlu ditangani sesuai kebutuhan analisis.

### Cleaning Data

Pada tahap cleaning, dilakukan penyesuaian format dan pembuatan fitur turunan agar analisis berbasis waktu lebih mudah:
- Memastikan kolom waktu bertipe datetime.
- Membuat fitur agregasi waktu seperti `date`, `year_month`, dan `hour`.
- Mengonversi kolom polutan menjadi numerik.


In [ ]:
air_df = air_df.copy()
air_df.columns = [str(c).strip() for c in air_df.columns]

cols_lower = {c.lower(): c for c in air_df.columns}
dt_col = cols_lower.get("datetime") or cols_lower.get("timestamp")
date_col = cols_lower.get("date")
time_col = cols_lower.get("time")

if dt_col is not None:
    air_df["datetime"] = pd.to_datetime(air_df[dt_col], errors="coerce")
elif date_col is not None and time_col is not None:
    air_df["datetime"] = pd.to_datetime(
        air_df[date_col].astype(str) + " " + air_df[time_col].astype(str), errors="coerce"
    )
else:
    candidate_cols = [c for c in air_df.columns if ("date" in c.lower()) or ("time" in c.lower())]
    candidate = candidate_cols[0] if candidate_cols else air_df.columns[0]
    air_df["datetime"] = pd.to_datetime(air_df[candidate], errors="coerce")

air_df = air_df.dropna(subset=["datetime"]).copy()
air_df["date"] = air_df["datetime"].dt.date
air_df["year_month"] = air_df["datetime"].dt.to_period("M").astype(str)
air_df["hour"] = air_df["datetime"].dt.hour

pollutant_cols = [
    c for c in air_df.columns
    if ("pm2.5" in c.lower()) or ("pm25" in c.lower()) or ("pm2_5" in c.lower()) or ("pm10" in c.lower())
    or (c.lower() in {"so2", "no2", "o3", "co"})
]
for c in pollutant_cols:
    air_df[c] = pd.to_numeric(air_df[c], errors="coerce")
    air_df.loc[air_df[c] == 999, c] = np.nan
    air_df.loc[air_df[c] < 0, c] = np.nan

air_df.head()

**Insight:**
- Kolom waktu sudah siap dipakai untuk agregasi harian/jam.
- Kolom polutan sudah dikonversi ke numerik sehingga mudah dihitung rata-ratanya.

## Exploratory Data Analysis (EDA)

Tahap EDA bertujuan untuk memahami pola umum pada data sebelum menjawab pertanyaan bisnis, seperti:
- Melihat ringkasan statistik untuk memahami sebaran dan rentang nilai.
- Mengecek korelasi antar variabel numerik sebagai indikasi hubungan awal.
- Mengidentifikasi variabel yang paling relevan untuk visualisasi explanatory.


### Ringkasan Statistik, Distribusi, dan Korelasi

In [ ]:
air_df.describe()

corr = air_df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="Blues", center=0)
plt.title("Korelasi Antar Variabel Numerik")
plt.tight_layout()
plt.show()

**Insight:**
- Korelasi antar polutan dapat memberi indikasi hubungan awal antar konsentrasi.
- Perhatikan missing value pada beberapa polutan dapat memengaruhi korelasi.

## Visualization & Explanatory Analysis

Pada tahap ini, visualisasi dibuat untuk menjawab pertanyaan bisnis secara jelas (explanatory), yaitu:
- Menunjukkan tren rata-rata harian PM2.5.
- Menampilkan pola rata-rata PM2.5 berdasarkan jam.
Output visualisasi diikuti ringkasan insight yang mengaitkan temuan dengan pertanyaan bisnis.


### Pertanyaan 1:
Bagaimana tren rata-rata harian PM2.5 dari waktu ke waktu?

In [ ]:
pm25_candidates = [c for c in air_df.columns if ("pm2.5" in c.lower()) or ("pm25" in c.lower()) or ("pm2_5" in c.lower())]
if not pm25_candidates:
    raise ValueError("Kolom PM2.5 tidak ditemukan pada dataset.")
pm25_col = pm25_candidates[0]

daily_df = (
    air_df.groupby("date", as_index=False)
    .agg(pm25_mean=(pm25_col, "mean"))
    .sort_values("date")
)

plt.figure(figsize=(14, 4))
sns.lineplot(data=daily_df, x="date", y="pm25_mean")
plt.title(f"Tren Rata-rata Harian {pm25_col}")
plt.xlabel("Tanggal")
plt.ylabel("Rata-rata")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

daily_df.head()

### Pertanyaan 2:
Pada jam berapa PM2.5 cenderung paling tinggi?

In [ ]:
hourly_df = (
    air_df.groupby("hour", as_index=False)
    .agg(pm25_mean=(pm25_col, "mean"))
    .sort_values("hour")
)

plt.figure(figsize=(12, 4))
sns.lineplot(data=hourly_df, x="hour", y="pm25_mean", marker="o")
plt.title(f"Rata-rata {pm25_col} berdasarkan Jam")
plt.xlabel("Jam")
plt.ylabel("Rata-rata")
plt.xticks(range(0, 24, 1))
plt.tight_layout()
plt.show()

hourly_df

**Insight:**
- Tren harian membantu melihat periode peningkatan/penurunan konsentrasi PM2.5.
- Pola per jam membantu mengidentifikasi jam-jam dengan konsentrasi PM2.5 yang cenderung lebih tinggi.

## Analisis Lanjutan (Opsional)

Bagian opsional ini berisi teknik analisis lanjutan tanpa algoritma machine learning, misalnya:
- **Manual grouping**: mengelompokkan jam menjadi kategori waktu (pagi/siang/sore/malam) untuk membandingkan pola PM2.5.
- **Binning (pd.cut / pd.qcut)**: membagi nilai PM2.5 ke beberapa kategori agar interpretasi lebih mudah.


In [ ]:
tmp = air_df.copy()

if "hour" in tmp.columns:
    time_bins = [-0.1, 5, 10, 14, 18, 23]
    time_labels = ["Dini Hari (0-5)", "Pagi (6-10)", "Siang (11-14)", "Sore (15-18)", "Malam (19-23)"]
    tmp["time_group"] = pd.cut(tmp["hour"], bins=time_bins, labels=time_labels, include_lowest=True)
    tg_df = (
        tmp.groupby("time_group", as_index=False)
        .agg(pm25_mean=(pm25_col, "mean"))
        .sort_values("time_group")
    )
    plt.figure(figsize=(10, 4))
    sns.barplot(data=tg_df, x="time_group", y="pm25_mean")
    plt.title(f"Rata-rata {pm25_col} berdasarkan Kelompok Waktu")
    plt.xlabel("Kelompok Waktu")
    plt.ylabel("Rata-rata")
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.show()

tmp = tmp.dropna(subset=[pm25_col]).copy()
tmp["pm25_bin"] = pd.qcut(tmp[pm25_col], q=4, duplicates="drop")
bin_df = tmp.groupby("pm25_bin", as_index=False).agg(
    count=(pm25_col, "count"),
    pm25_mean=(pm25_col, "mean"),
)
bin_df

plt.figure(figsize=(10, 4))
sns.barplot(data=bin_df, x="pm25_bin", y="count")
plt.title(f"Jumlah Pengamatan berdasarkan Kuartil {pm25_col}")
plt.xlabel(f"Kuartil {pm25_col}")
plt.ylabel("Jumlah Pengamatan")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()

## Conclusion

- Tren rata-rata harian PM2.5 menunjukkan periode peningkatan/penurunan kualitas udara.
- Pola per jam membantu mengidentifikasi jam dengan kecenderungan PM2.5 lebih tinggi.